<a href="https://colab.research.google.com/github/ernestomancebo/DeepLearningInPractice/blob/main/issa_cat_or_issa_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Issa cat or Issa Dog
This notebook atteps to do a binary classification for Cats and Dogs images from Kaggle datasets using a ConvNet.